In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import time

# 1. Configuration and Ticker Setup
ticker = "RSP"
start_date = "2003-05-01"

print(f"Starting data download for {ticker}...")
start_time = time.time()

# 2. Download RSP data with progress bar enabled
# 'progress' shows the yfinance internal progress bar
data = yf.download(ticker, start=start_date, progress=True)

end_time = time.time()
download_duration = end_time - start_time

print(f"Download completed in {download_duration:.2f} seconds.")

# 3. Prepare the Target variable
# Objective: Predict if the price will go UP tomorrow.
# Target: 1 if tomorrow's Close > today's Close, else 0.
data["Tomorrow"] = data["Close"].shift(-1)
data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)

# 4. Data Cleaning
# Remove the last row as it lacks a 'Tomorrow' price to compare
data = data.dropna()

print(f"Dataset successfully prepared with {data.shape[0]} samples.")
data.head()

In [ ]:
import matplotlib.pyplot as plt

# Set a professional visual style
plt.style.use('seaborn-v0_8-grid')

# Create the plot
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], color='#1f77b4', linewidth=1.5, label='RSP Close Price')

# Enhance chart with metadata
plt.title(f'Historical Price Trend: {ticker} (Equal Weight S&P 500)', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Adjusted Close Price (USD)', fontsize=12)

# Highlight key info
plt.legend(loc='upper left')
plt.annotate(f'Last Price: ${data["Close"].iloc[-1]:.2f}', 
             xy=(data.index[-1], data['Close'].iloc[-1]),
             xytext=(data.index[-1] - pd.Timedelta(days=1000), data['Close'].max()),
             arrowprops=dict(facecolor='black', shrink=0.05, width=1))

plt.tight_layout()
plt.show()